In [46]:
import datetime
import json
from dateutil.relativedelta import relativedelta
from django.db.models.functions import TruncDay, Trunc, TruncMonth
from django.db.models import Count
from django.core.serializers.json import DjangoJSONEncoder
from django.http import JsonResponse
from apps.scg_app import models

In [40]:
today = datetime.date.today()
date_ini = today - relativedelta(months=+6, days=+today.day-1)
date_end = today - relativedelta(days=+today.day)

date_ini, date_end

(datetime.date(2020, 1, 1), datetime.date(2020, 6, 30))

In [41]:
filtered_classes = models.Clase.objects.filter(
    fecha__gte=date_ini,
    fecha__lte=date_end
)

In [81]:
types_filters = {
    "Total": None,
    "Realizadas": {"estado": "0"},
    "Reemplazos": {"reemplazo__isnull": False},
    "Ausencias": {"ausencia__isnull": False},
}
chart_results = {}

In [82]:
for _type, _filter in types_filter.items():
    work_classes = filtered_classes
    if _filter:
        work_classes = filtered_classes.filter(**_filter)
    
    chart_data = work_classes.annotate(
        date=TruncMonth("fecha")
    ).values(
        "date"
    ).annotate(
        y=Count("id")
    ).order_by("-date")
    
    chart_results[_type] = json.dumps(list(chart_data), cls=DjangoJSONEncoder)

In [83]:
jr = JsonResponse({"results": chart_results})
jr.content

b'{"results": {"Total": "[{\\"date\\": \\"2020-06-01\\", \\"y\\": 61}, {\\"date\\": \\"2020-05-01\\", \\"y\\": 74}, {\\"date\\": \\"2020-04-01\\", \\"y\\": 88}, {\\"date\\": \\"2020-03-01\\", \\"y\\": 87}, {\\"date\\": \\"2020-02-01\\", \\"y\\": 40}]", "Realizadas": "[{\\"date\\": \\"2020-06-01\\", \\"y\\": 58}, {\\"date\\": \\"2020-05-01\\", \\"y\\": 65}, {\\"date\\": \\"2020-04-01\\", \\"y\\": 83}, {\\"date\\": \\"2020-03-01\\", \\"y\\": 84}, {\\"date\\": \\"2020-02-01\\", \\"y\\": 35}]", "Reemplazos": "[{\\"date\\": \\"2020-06-01\\", \\"y\\": 2}, {\\"date\\": \\"2020-04-01\\", \\"y\\": 1}, {\\"date\\": \\"2020-03-01\\", \\"y\\": 1}, {\\"date\\": \\"2020-02-01\\", \\"y\\": 1}]", "Ausencias": "[{\\"date\\": \\"2020-05-01\\", \\"y\\": 9}, {\\"date\\": \\"2020-04-01\\", \\"y\\": 3}, {\\"date\\": \\"2020-03-01\\", \\"y\\": 2}, {\\"date\\": \\"2020-02-01\\", \\"y\\": 4}]"}}'